In [1]:
%%html
<style>
    body {
        --vscode-font-family: "Segoe UI"
    }
</style>

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
from glob import glob
from pathlib import Path
from llama_index.readers.file.flat_reader import FlatReader
from llama_index.node_parser.file import SimpleFileNodeParser
from llama_index.node_parser import SentenceSplitter
from llama_index import VectorStoreIndex, set_global_handler
from llama_index.ingestion import IngestionPipeline

In [4]:
set_global_handler("simple")

In [5]:
DATAROOT = Path.home() / "mldata" / "sherlock" / "md"

In [6]:
reader = FlatReader()
parser = SimpleFileNodeParser()
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=128)

In [7]:
all_nodes = []
for filepath in glob(str(DATAROOT/"*.md")):
    docs = reader.load_data(Path(filepath))
    nodes = parser.get_nodes_from_documents(docs)
    chunks = splitter.get_nodes_from_documents(nodes)
    print(f"Split {filepath} into {len(chunks)} chunks.")
    all_nodes += chunks

Split /Users/avilay/mldata/sherlock/md/retn.md into 175 chunks.
Split /Users/avilay/mldata/sherlock/md/case.md into 133 chunks.
Split /Users/avilay/mldata/sherlock/md/lstb.md into 113 chunks.
Split /Users/avilay/mldata/sherlock/md/advs.md into 165 chunks.
Split /Users/avilay/mldata/sherlock/md/mems.md into 134 chunks.


In [8]:
len(all_nodes)

720

In [16]:
all_nodes[0]

TextNode(id_='1cb4de75-6b97-4818-85e0-b2983cf6930b', embedding=None, metadata={'Header 1': 'The Return of Sherlock Holmes', 'filename': 'retn.md', 'extension': '.md'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d3d82be7-0864-4fc7-a151-f3896d819d43', node_type=<ObjectType.TEXT: '1'>, metadata={'Header 1': 'The Return of Sherlock Holmes', 'filename': 'retn.md', 'extension': '.md'}, hash='d6e9156f0b85bdc691f05a7aa71118c825424240dbba396317a5198e4325725d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='0902d526-25c3-4b88-aa03-c0ca32c267a4', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='6ba08607b54ee3be8623afad2eda4a1324ecb912cba3eac808066d39dbf28354')}, text='The Return of Sherlock Holmes\nArthur Conan Doyle', start_char_idx=0, end_char_idx=48, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

```python
TextNode(
    id_='1cb4de75-6b97-4818-85e0-b2983cf6930b', 
    embedding=None, 
    metadata={
        'Header 1': 'The Return of Sherlock Holmes', 
        'filename': 'retn.md', 
        'extension': '.md'
    }, 
    excluded_embed_metadata_keys=[], 
    excluded_llm_metadata_keys=[], 
    relationships={
        <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
            node_id='d3d82be7-0864-4fc7-a151-f3896d819d43', 
            node_type=<ObjectType.TEXT: '1'>, 
            metadata={
                'Header 1': 'The Return of Sherlock Holmes', 
                'filename': 'retn.md', 
                'extension': '.md'
            }, 
            hash='d6e9156f0b85bdc691f05a7aa71118c825424240dbba396317a5198e4325725d'
        ), 
        <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(
            node_id='0902d526-25c3-4b88-aa03-c0ca32c267a4', 
            node_type=<ObjectType.TEXT: '1'>, 
            metadata={}, 
            hash='6ba08607b54ee3be8623afad2eda4a1324ecb912cba3eac808066d39dbf28354'
        )
    }, 
    text='The Return of Sherlock Holmes\nArthur Conan Doyle', 
    start_char_idx=0, 
    end_char_idx=48, 
    text_template='{metadata_str}\n\n{content}', 
    metadata_template='{key}: {value}', 
    metadata_seperator='\n'
)
```

In [9]:
index = VectorStoreIndex(all_nodes)

In [17]:
index.ref_doc_info

{'d3d82be7-0864-4fc7-a151-f3896d819d43': RefDocInfo(node_ids=['1cb4de75-6b97-4818-85e0-b2983cf6930b'], metadata={'Header 1': 'The Return of Sherlock Holmes', 'filename': 'retn.md', 'extension': '.md'}),
 'b4e5976c-c194-4474-ae76-c38d7c671c44': RefDocInfo(node_ids=['0902d526-25c3-4b88-aa03-c0ca32c267a4', 'a7d1c166-d5f5-458f-ad3c-00357aaacb22', '5f84f9ad-643a-4471-a0b9-da874ec9fd64', '5fdfe494-4d3c-4327-a03d-ba66aed532a0', 'a4f4ba15-8592-42e7-bd60-90366d7af9c5', '19a7c907-b1dc-4cd5-b1d9-d65d3615a6bf', 'af3390d2-fc78-40b6-bccf-07c761cbffe3', '5d8b5531-35d2-4588-9ed6-248a1b063af6', '933043c8-744a-4364-95d3-0c30a218f8e7', '7c0af202-5d00-4d9f-b2c3-b4b99e4cb852', '1df9c17a-adbc-4ea6-8f60-25aeee703c6f', '815a6417-f2b5-40fc-8df1-38e4cb0f0a26', '1f290ab2-ad92-4031-88dc-c472fc73b151'], metadata={'Header 1': 'The Return of Sherlock Holmes', 'Header 2': 'The Adventure of the Empty House', 'filename': 'retn.md', 'extension': '.md'}),
 '966950f8-cbb9-427b-8a1e-73e2edf718c6': RefDocInfo(node_ids=['a7c

In [18]:
doc_id = "57f40ee8-6b68-48dd-a611-f83c3248d7af"
for node in all_nodes:
    if node.id_ == doc_id:
        print(node)

In [19]:
node_id = "dd18da4f-0949-4b24-b841-7394d3890adb"
for node in all_nodes:
    if node.id_ == node_id:
        print(node)

Node ID: dd18da4f-0949-4b24-b841-7394d3890adb
Text: "'On the contrary,' I answered, 'I think it is fairly evident
that I do.  Pray take a chair.  I can spare you five minutes if you
have anything to say.'   "'All that I have to say has already crossed
your mind,' said he.   "'Then possibly my answer has crossed yours,' I
replied.   "'You stand fast?'   "'Absolutely.'   "He clapped his hand
into h...


In [10]:
query_engine = index.as_query_engine()
query_result = query_engine.query("How does Sherlock Holmes feel about Irene Adler?")
print(query_result)

** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
Header 1: The Adventures of Sherlock Holmes
Header 2: A Scandal in Bohemia
Header 3: CHAPTER I 
filename: advs.md
extension: .md

CHAPTER I 
To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has

In [11]:
query_result = query_engine.query("Who stole the blue carbuncle?")
print(query_result)

** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
Header 1: The Adventures of Sherlock Holmes
Header 2: The Adventure of the Blue Carbuncle
filename: advs.md
extension: .md

Sherlock Holmes had opened his mouth to reply, when the door flew open, and Peterson, the commissionaire, rushed into the apartment with flushed cheeks and the face of a man who is dazed with astonishment. 

"The goose, Mr. Holmes! The goose, sir!" he gasped. 

"Eh? What of it, then? Has it returned to life and flapped off through the kitchen window?" Holmes twisted himself round upon the sofa to get a fairer view of the man's excited fa

In [12]:
query_result = query_engine.query("Who killed Enoch Drebber?")

** Messages: **
system: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
user: Context information is below.
---------------------
Header 1: The Case Book of Sherlock Holmes
Header 2: The Adventure of the Creeping Man
filename: case.md
extension: .md

In all our adventures I do not know that I have ever seen a more strange sight than this impassive and still dignified figure crouching frog-like upon the ground and goading to a wilder exhibition of passion the maddened hound, which ramped and raged in front of him, by all manner of ingenious and calculated cruelty. 

And then in a moment it happened! It was not the chain that broke, but it was the collar that slipped, for it had been made

In [13]:
print(query_result)

James Barclay was responsible for the death of Enoch Drebber.


In [ ]:
transformations = [parser, splitter]
pipeline = IngestionPipeline(transformations=transformations)